In [1]:
import sys
sys.path.append('/Users/mariana/Documents/research/xlstm-jax')
from xlstm_jax.models.xlstm_clean.blocks.mlstm.backend import  recurrent_step_stabilized_simple
from xlstm_jax.models.xlstm_clean.blocks.mlstm.layer import mLSTMLayerConfig

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jax
import jax.numpy as jnp
import numpy as np
import math
import flax.linen as nn
from dataclasses import dataclass, field

In [4]:
import sys
sys.path.append('/Users/mariana/Documents/research/nanoTempoPFN')
# sys.path.append('/Users/mariana/Documents/research/nanoTempoPFN/src')

from pathlib import Path
from src.data.containers import ShardedDataset, DataLoader
from src.data.time_features import compute_batch_time_features

sys.path.append('/Users/mariana/Documents/research/nanoTempoPFN/src')
from model.recurrent_lstm_layer import mLSTMWeavingLayer


In [8]:
# dataset = ShardedDataset(root_path=Path('../data'))
# loader = DataLoader(dataset, batch_size=8, future_length=64)

# batch = next(loader)
# print(f"history: {batch.history.shape}, future: {batch.future.shape}")


In [9]:
# history_tf, future_tf = compute_batch_time_features(
#     start=batch.start,
#     history_length=batch.history_length,
#     future_length=batch.future_length,
#     batch_size=batch.batch_size,
#     frequency=batch.frequency,
# )
# print(f"history_tf: {history_tf.shape}, future_tf: {future_tf.shape}")


In [ ]:
from model.recurrent_lstm_cell import mLSTMWeavingCellConfig

# ===============================================
# Super important: _inner_embedding_dim must be 
# DH*NH. TODO: now you can implement the weaving
# layer below.
# ===============================================

# Config
input_dim = 10
B, S, DH = 2, 5, 8
NH = 3
config = mLSTMLayerConfig(embedding_dim=DH, num_heads=NH)
config._inner_embedding_dim = DH*NH
config.mlstm_cell = mLSTMWeavingCellConfig(embedding_dim=DH, num_heads=config.num_heads)

# Test
x = jax.random.normal(jax.random.PRNGKey(0), (B, S, input_dim))
c_state = jnp.zeros((B, NH, DH, DH))
n_state = jnp.zeros((B, NH, DH, 1))
m_state = jnp.zeros((B, NH, 1, 1))

layer = mLSTMWeavingLayer(config=config)
params = layer.init(jax.random.PRNGKey(1), x, c_state, n_state, m_state)

y, (c, n, m) = layer.apply(params, x, c_state, n_state, m_state)

print(f"Input: {x.shape} -> Output: {y.shape}")

 q shape is (2, 5, 24) 



SHAPES igate_preact, fgate_preact = (2, 5, 3), (2, 5, 3)
 q shape is (2, 5, 24) 



SHAPES igate_preact, fgate_preact = (2, 5, 3), (2, 5, 3)
Input: (2, 5, 10) -> Output: (2, 5, 8)


In [18]:
config._inner_embedding_dim

24

In [ ]:
@dataclass
class ModelConfig:
    embedding_dim: int = 32
    n_layers: int = 4
    layer_config: mLSTMLayerConfig = field(default_factory=lambda: mLSTMLayerConfig(embedding_dim=32))

class WeavingLSTM:

    def __init__(self, config: ModelConfig):
        self.config = config

    @nn.compact
    def __call__(self, x, h, training=True):
        
        for i in range(self.config.n_layers):
            x, (c_state, n_state, m_state) = mLSTMWeavingLayer(config=self.config.layer_config)(x, h, training)
